## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

In [ ]:
# For installing the libraries & downloading models from HF Hub
!pip install huggingface_hub==0.30.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.4.0 numpy==1.25.2 -q

In [ ]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import numpy as np
import pandas as pd

## Question Answering using LLM

#### Downloading and Loading the model

- Number of pages in the document

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

## Calling llama gives blas=1, GPU is detected
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 |



In [ ]:
#uncomment the below snippet of code if the runtime is connected to GPU.
llm_mistral = Llama(
    model_path=model_path,
    n_ctx=2300,
    n_gpu_layers=38,
    n_batch=512
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### Response

In [ ]:
def response(query,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    model_output = llm_mistral(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
query='What is the protocol for managing sepsis in a critical care unit?'
response(query)

'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. Resusc'

- Looks like max tokens is limited given the length of the answers, we will increase going forward

In [ ]:
query='What is the protocol for managing sepsis in a critical care unit?'
response(query, max_tokens=500)

Llama.generate: prefix-match hit


'\n\nSepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are general steps for managing sepsis in a critical care unit:\n\n1. Early recognition: Recognize the signs and symptoms of sepsis early and initiate treatment as soon as possible. Sepsis can present with various clinical features, including fever or hypothermia, tachycardia or bradycardia, altered mental status, respiratory distress, and lactic acidosis.\n2. Source control: Identify and address the source of infection as quickly as possible. This may involve surgical intervention, such as drainage of an abscess or debridement of necrotic tissue.\n3. Fluid resuscitation: Administer intravenous fluids to maintain adequate blood pressure and organ perfusion. The goal is to achieve a mean arterial pressure (MAP) of at least 65 mmHg and a central venous oxygen saturation (ScvO2) of greater than 70%.\n4. Vasopressor

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
query='What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?'
response(query)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain may start as a mild discomfort around the navel or in the lower right abdomen, which then gradually moves to the right lower quadrant and becomes more severe over time. The pain may be constant or intermittent and is often worsened by movement, coughing, or deep breathing'

In [ ]:
query='What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?'
response(query,max_tokens=500)

Llama.generate: prefix-match hit


'\n\nAppendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The symptoms of appendicitis can vary from person to person, but some common signs include:\n\n1. Abdominal pain: The pain typically starts as a mild discomfort around the navel or in the lower right abdomen. It may then move to the right lower quadrant and become more severe over time. The pain is often described as sharp, crampy, or constant.\n2. Loss of appetite: People with appendicitis may lose their appetite due to abdominal discomfort or nausea.\n3. Nausea and vomiting: Vomiting is a common symptom of appendicitis, especially in the later stages of the condition.\n4. Fever: A fever of 100.4°F (38°C) or higher may be present in some cases of appendicitis.\n5. Diarrhea or constipation: Some people with appendicitis experience diarrhea, while others have constipation.\n6. Rebound tenderness: When pressure is applied to the

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
query='What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?'
response(query,max_tokens=500)

Llama.generate: prefix-match hit


"\n\nSudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can result in round or oval bald patches on the scalp, but it can also occur on other parts of the body such as the beard area, eyebrows, or eyelashes.\n\nThe exact cause of alopecia areata is not known, but it's believed to be related to a problem with the immune system. Some possible triggers for this condition include stress, genetics, viral infections, and certain medications.\n\nThere are several treatments that have been shown to be effective in addressing sudden patchy hair loss:\n\n1. Corticosteroids: These are anti-inflammatory drugs that can help reduce inflammation and suppress the immune system's attack on the hair follicles. They can be applied topically or taken orally, depending on the severity of the condition.\n2. Minoxidil: This is a medication that has been shown to promote hair growth in some people with alopecia areata. It works by increa

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
query='What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?'
response(query,max_tokens=500)

Llama.generate: prefix-match hit


"\n\nA person who has sustained a physical injury to brain tissue, also known as a traumatic brain injury (TBI), may require various treatments depending on the severity and location of the injury. Here are some common treatments recommended for TBIs:\n\n1. Emergency care: The first priority is to ensure the person's airway is clear, they are breathing, and their heart is beating normally. In severe cases, emergency surgery may be required to remove hematomas or other obstructions.\n2. Medications: Depending on the symptoms, medications may be prescribed to manage conditions such as swelling, seizures, pain, or infections. For example, corticosteroids may be used to reduce brain swelling, and anticonvulsants may be given to prevent seizures.\n3. Rehabilitation: Rehabilitation is an essential part of the recovery process for TBI patients. Rehabilitation may include physical therapy to help with mobility and strength, occupational therapy to help with daily living skills, speech therapy 

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
query='What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?'
response(query,max_tokens=500)

Llama.generate: prefix-match hit


"\n\nFirst and foremost, if you suspect that someone has fractured their leg while hiking, it's essential to ensure their safety and prevent further injury. Here are some necessary precautions:\n\n1. Keep the person calm and still: Encourage them to remain as still as possible to minimize pain and prevent worsening the injury.\n2. Assess the situation: Check for any signs of shock, such as pale skin, rapid heartbeat, or shallow breathing. If you notice these symptoms, seek medical help immediately.\n3. Immobilize the leg: Use a splint, sling, or other available materials to immobilize the leg and prevent movement. Be sure not to apply too much pressure on the injury site.\n4. Provide pain relief: Offer over-the-counter pain medication, such as acetaminophen or ibuprofen, to help manage pain.\n5. Seek medical attention: If the fracture is severe or if you suspect that there may be other injuries, seek medical help as soon as possible.\n\nOnce you've ensured the person's safety and stabi

We see there's lot more relevant content being given for the response when max tokens is increased

## Question Answering using LLM with Prompt Engineering

In [ ]:
def generate_pe_response(user_input,qna_pe_system_message,max_tokens=500,temperature=0,top_p=0.95,top_k=50):
    # global qna_pe_system_message

    prompt = f"""[INST]{qna_pe_system_message}\n
                {'user'}: {user_input}
                [/INST]"""

    # Generate the response
    try:
        response = llm_mistral(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
query_pe_1="What is the protocol for managing sepsis in a critical care unit?"
qna_pe_system_message = "You are assistant expert in diseases and handling treatment of patients in the critical care unit. User questions will begin with the token: 'User'. If you do not understand or know the answer please respond 'I don't know'"

response_pe_1=generate_pe_response(query_pe_1,qna_pe_system_message)
response_pe_1

Llama.generate: prefix-match hit


"I'm an assistant expert in diseases and handling treatment of patients in the critical care unit. In managing sepsis in a critical care unit, the following steps are typically taken:\n\n1. Early recognition and diagnosis: Identify sepsis early based on clinical signs and laboratory results such as fever, tachycardia, respiratory distress, lactate levels, and white blood cell count.\n2. Immediate fluid resuscitation: Administer intravenous fluids to maintain adequate tissue perfusion and restore circulating volume. The goal is to achieve a mean arterial pressure (MAP) of 65 mmHg or higher and a central venous oxygen saturation (ScvO2) of greater than 70%.\n3. Antibiotic therapy: Start broad-spectrum antibiotics as soon as possible based on the suspected source of infection and local microbiology data.\n4. Source control: Identify and address the underlying source of infection, such as removing an infected catheter or draining an abscess.\n5. Vasopressor support: If fluid resuscitation 

### Compared to the plain user prompt without system message, with prompt engineering misses on the point,
"6. Glucose control: Maintain strict glucose control to prevent hyperglycemia, which can worsen inflammation and organ damage. A target blood glucose level of 140-180 mg/dL is recommended."
But adds other points such as, monitoring and multidisciplinary care


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
query_pe_2= "Please answer the following with as much detail as possible:\
            - What are the common symptoms for appendicitis?\
            - Can it be cured via medicine?\
            - If not, what surgical procedure should be followed to treat it?"
qna_pe_system_message = "You are assistant expert in diseases and handling treatment of patients in the critical care unit. User questions will begin with the token: 'User'. If you do not understand or know the answer please respond 'I don't know'"

response_pe_2=generate_pe_response(query_pe_2,qna_pe_system_message)
response_pe_2

Llama.generate: prefix-match hit


'Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. Here are answers to your questions:\n\n- Common symptoms for appendicitis include:\n  - Sudden pain in the lower right abdomen that begins around the navel and then shifts to the right.\n  - Loss of appetite.\n  - Nausea and vomiting.\n  - Fever, which may be low-grade at first but can rise as high as 103°F (39.4°C) or more.\n  - Abdominal swelling and rigidity.\n  - Pain upon walking, coughing, or making other jarring movements.\n  - Inability to pass gas or have a bowel movement.\n\n- Appendicitis cannot be cured via medicine alone. Antibiotics may be used to treat any accompanying infection, but they do not address the underlying inflammation in the appendix. Surgery is the only effective treatment for appendicitis.\n\n- The standard surgical procedure for treating appendicitis is an appendectomy. During this procedure, the sur

### With prompt engineering, the symptoms are touched upon elaborating where needed. More importance is given to treatment and risks involved. Some of the content in the treatment and risks was not in the response that had just the prompt(without prompt engineering)

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
query_pe_3= "Please answer the following:\
            - What are the effective treatments or solutions for \
            addressing sudden patchy hair loss, commonly seen \
            as localized bald spots on the scalp \
            - What could be the possible causes behind it?"
qna_pe_system_message = "You are assistant expert in dermatology. User questions will begin with the token: 'User'. If you do not understand or know the answer please respond 'I don't know'"

response_pe_3=generate_pe_response(query_pe_3,qna_pe_system_message)
response_pe_3

Llama.generate: prefix-match hit


'I. Possible Causes of Sudden Patchy Hair Loss:\n\n1. Alopecia Areata: An autoimmune disorder that causes hair loss in small, round patches on the scalp, beard or other areas of the body.\n2. Trauma or stress: Physical or emotional stress can cause sudden hair loss, often referred to as telogen effluvium.\n3. Nutritional deficiencies: Lack of essential nutrients like iron, zinc, and vitamin B12 can lead to hair loss.\n4. Hormonal imbalances: Thyroid disorders or other hormonal issues can cause patchy hair loss.\n5. Medications: Certain medications such as chemotherapy drugs, antidepressants, and beta-blockers can cause hair loss.\n6. Infections: Fungal infections like ringworm or bacterial infections can lead to patchy hair loss.\n7. Other medical conditions: Conditions like lupus, diabetes, and lichen planus can cause hair loss.\n\nII. Effective Treatments for Sudden Patchy Hair Loss:\n\n1. Topical treatments: Minoxidil (Rogaine) is a topical medication that can help stimulate hair gr

### With prompt engineering, the causes are more detailed than using just the query. For the treatments, prompt engineered query missed out on DHT blockers and the response was succint.

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
query_pe_4= "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
qna_pe_system_message = "You are assistant expert at human brain domain regarding diseases, injuries,symptoms,treatments, emergency care. User questions will begin with the token: 'User'. If you do not understand or know the answer please respond 'I don't know'"

response_pe_4=generate_pe_response(query_pe_4,qna_pe_system_message)
response_pe_4

Llama.generate: prefix-match hit


"The treatment for a brain injury can depend on the severity and location of the injury. Here are some common treatments:\n\n1. Emergency care: If the injury is recent, the first priority is to ensure the person's airway is clear, they are breathing, and have adequate circulation. This may involve CPR or intubation if necessary.\n2. Medications: Depending on the symptoms, medications may be prescribed to manage conditions such as swelling, seizures, or infections. For example, corticosteroids may be used to reduce swelling, anticonvulsants for seizure control, and antibiotics for infections.\n3. Surgery: In some cases, surgery may be necessary to remove blood clots or repair damaged tissue. This is usually only considered for more severe injuries.\n4. Rehabilitation: Regardless of the severity of the injury, rehabilitation is an essential part of the recovery process. This can include physical therapy to help regain motor function, occupational therapy to learn new skills, and speech t

### This response was mostly similar to one without promt engineering. With prompt engineering response inlcuded alternative therapies as well, which is good to be considered

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
query_pe_5= "Please answer the following:\
            - What are the necessary precautions for a person who has fractured their leg during a hiking trip \
            - What are the treatment steps \
            - What should be considered for their care and recovery?"
qna_pe_system_message = "You are assistant expert at human body, its functionality, including diseases, injuries,symptoms,treatments, medication,surgeries, emergency care. You are also a care and recovery specialist. User questions will begin with the token: 'User'. If you do not understand or know the answer please respond 'I don't know'"

response_pe_5=generate_pe_response(query_pe_5,qna_pe_system_message)
response_pe_5

Llama.generate: prefix-match hit


'User: What are the necessary precautions for a person who has fractured their leg during a hiking trip?\n\nI. Ensure the safety of the injured person by preventing further injury. This may include moving them to a safe location, keeping them warm, and providing shelter from the elements.\nII. Do not attempt to move the person with a fracture unless it is necessary for their safety or to prevent further harm. If you must move them, try to keep the leg as still as possible.\nIII. Apply a splint or immobilize the leg using a makeshift sling if available. This will help reduce pain and prevent the bone from moving, which can worsen the injury.\nIV. Keep the injured person comfortable by providing them with water, food, and any necessary medications.\nV. Do not give them alcohol or caffeine as these substances can increase pain and interfere with their ability to heal.\nVI. If possible, contact emergency medical services for further assistance.\n\nUser: What are the treatment steps for a f

### The response with prompt engineering was elaborate stressing on care and recovery treatments. It also gave good insights into all aspects for each question.

## Data Preparation for RAG

### Loading the Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pdf_path ='/content/drive/MyDrive/Colab Notebooks/medical_diagnosis_manual.pdf'

In [ ]:
pdf_loader = PyMuPDFLoader(pdf_path)
pdf_loaded = pdf_loader.load()

In [ ]:
pdf_loaded[70].page_content

### Data Overview

#### Checking the first 5 pages

In [ ]:
for i in range(5):
    print(f'Page no-{i+1}\n')
    print(pdf_loaded[i].page_content,end='\n')

Page no-1

malathiv1239@gmail.com
EK1STUWL8Q
ant for personal use by malathiv1239@g
shing the contents in part or full is liable 

Page no-2

malathiv1239@gmail.com
EK1STUWL8Q
This file is meant for personal use by malathiv1239@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.

Page no-3

Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................
53
1 - Nut

#### Checking the number of pages

In [ ]:
len(pdf_loaded)

4114

### Data Chunking

### For chunk_overlap, we tried different sizes, 40 works best

In [ ]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap= 40
)

In [ ]:
document_chunks = pdf_loader.load_and_split(text_splitter)

In [ ]:
len(document_chunks)

8618

In [ ]:
for i in range(5):
    print(f'chunk no:{i}\n')
    print(f'{document_chunks[i].page_content}\n\n')

chunk no:0

malathiv1239@gmail.com
EK1STUWL8Q
ant for personal use by malathiv1239@g
shing the contents in part or full is liable


chunk no:1

malathiv1239@gmail.com
EK1STUWL8Q
This file is meant for personal use by malathiv1239@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.


chunk no:2

Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................
53
1 -

In [ ]:
document_chunks[0].page_content

'malathiv1239@gmail.com\nEK1STUWL8Q\nant for personal use by malathiv1239@g\nshing the contents in part or full is liable'

In [ ]:
document_chunks[-2].page_content

'Y\nYaws 1266-1267\nforest 1379\nY chromosome 3373 (see also Genetic)\nabnormalities of 3005\nYeast infection (see also Fungal infection)\nvaginal 2542, 2544, 2545\nYellow fever 1400, 1429, 1437\nhepatic inflammation in 248\nvaccine against 1172, 1437, 3441\nYellow nail syndrome 732, 1995\npleural effusion in 1997\nYellow skin (see Jaundice)\nYersinia infection 1167, 1256-1257\nY. enterocolitica infection 147\nY. pestis infection 1924\nYew poisoning 3338\nYips 1762\nYo, antibodies to 1056\nYolk sac tumor 2476\nThe Merck Manual of Diagnosis & Therapy, 19th Edition\nY\n4103\nmalathiv1239@gmail.com\nEK1STUWL8Q\nThis file is meant for personal use by malathiv1239@gmail.com only.\nSharing or publishing the contents in part or full is liable for legal action.'

### Embedding

In [ ]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

In [ ]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

In [ ]:
print("Dimension of the embedding vector ",len(embedding_1))
len(embedding_1)==len(embedding_2)

Dimension of the embedding vector  1024


True

### Vector Database

In [ ]:
out_dir = 'medical_diag_manual'

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [ ]:
vectorstore = Chroma.from_documents(
    document_chunks,
    embedding_model,
    persist_directory=out_dir
)

In [ ]:
vectorstore = Chroma(persist_directory=out_dir,embedding_function=embedding_model)

In [ ]:
vectorstore.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='thenlper/gte-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [ ]:
vectorstore.similarity_search("sinus diseases are contagious ",k=3)

### Retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

In [ ]:
rel_docs = retriever.get_relevant_documents("Common diseases that can be cured")
rel_docs

- The ouput discusses the diagnosis and treatment of common diseases

### System and User Prompt Template

In [ ]:
qna_system_message = """
You are an assistant whose work is to review the report and provide the appropriate answers from the context.
User input will have the context required by you to answer user questions.
This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Please answer only using the context provided in the input. Do not mention anything about the context in your final answer.

If the answer is not found in the context, respond "I don't know".
"""

In [ ]:
qna_user_message_template = """
###Context
Here are some documents that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

### Response Function

In [ ]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm_mistral(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
user_input='What is the protocol for managing sepsis in a critical care unit?'
generate_rag_response(user_input)

Llama.generate: prefix-match hit


'Answer:\nPatients with septic shock should be treated in an ICU. The following should be monitored frequently: systemic pressure; CVP, PAOP, or both; pulse oximetry; ABGs; blood glucose, lactate, and electrolyte levels; renal function; and possibly sublingual PCO2. Urine output should be measured, usually with an indwelling catheter. Fluid resuscitation with 0.9% saline should be given until CVP reaches 8 mm Hg (10 cm H2O)'

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
user_input='What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?'
generate_rag_response(user_input)

Llama.generate: prefix-match hit


'###Answer\nThe common symptoms for appendicitis include abdominal pain, anorexia, and abdominal tenderness. Appendicitis cannot be cured via medicine alone; surgery, specifically a surgical removal of the appendix called appendectomy, is required for treatment.'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
query3='What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?'
generate_rag_response(query3)

Llama.generate: prefix-match hit


'Based on the context provided, the condition being described is Alopecia Areata. The effective treatments mentioned include Minoxidil (2% for women, 2% or 5% for men) and Finasteride (1 mg po once/day). These treatments can help prolong the anagen growth phase and enlarge miniaturized follicles into mature terminal hairs, as well as inhibit the conversion of testosterone to dihydrotestosterone, respectively. The context also suggests that scalp biopsy may be necessary for definitive diagnosis when alopecia persists. Poss'

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
query4='What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?'
generate_rag_response(query4)

Llama.generate: prefix-match hit


'Based on the context provided, the recommended treatments for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function include:\n\n1. Ensuring a reliable airway and maintaining adequate ventilation, oxygenation, and blood pressure.\n2. Surgery to place monitors to track and treat intracranial pressure, decompress the brain if intracranial pressure is increased, or remove intracranial hematomas.\n3. Rehabilitation, which includes prevention of secondary disabilities such as pressure ulcers, joint contractures, and p'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
query5='What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?'
generate_rag_response(query5)

Llama.generate: prefix-match hit


'Based on the context provided, here is the answer:\n\nThe person with a fractured leg should seek medical care at once. In the meantime, they can apply ice to the affected area to help reduce swelling. The leg should be immobilized using a splint or cast to prevent movement and allow the bone to heal properly. Elevating the leg above heart level for the first 48 hours after the injury can also help reduce swelling and promote healing. If the person is unable to walk, they may need to use crutches to stay off their feet for several days. Good hygiene is'

### Fine-tuning

### Parameters to consider:
### Greedy sampling vs Random sampling
- Greedy sampling (Top-K=1) leads to coherent but uncreative outputs like always ordering the single most popular/likely menu item
- Random sampling enables maximum creativity but outputs are incoherent like ordering by pulling any menu item out of a hat randomly

### temperature
- Range 0-1. 0-deterministic, 1-creative

### top_p
- The top_p controls the model output by augmenting the vocabulary size as only those tokens are considered for which the cumulative probability is greater than the top_p value (the cumulative probability is calculated by sorting the tokens by their probability in a descending order and then adding them up).
 - close to 1 (e.g. 0.95): Output more random. A large portion of the vocabulary is considered, making the output more random. This is because even tokens with relatively low probabilities are included in the nucleus from which we’re sampling.
 - close to 0 (e.g. 0.1): Output more deterministic.Only the very top tokens (those with the highest probabilities) are considered, making the output more deterministic.

 ### top_k
- This sorts tokens in descending order by their probabilites, re-calculate probabilities (using softmax on logits) on reduced vocabulary


## Explanation on params

### Temperature
The language model first computes the unnormalized log probability scores for each word in the vocabulary given the previous context.
These log probabilities are then divided by the temperature value: log_prob_scaled = log_prob / temperature
If temperature < 1, this has the effect of making the log probabilities more extreme (i.e. higher log probs become higher, lower ones become lower)
If temperature > 1, it has the opposite effect of making the log probabilities less extreme and shrinking towards 0.
After scaling by temperature, a softmax function is applied to convert these scaled log probabilities into a proper probability distribution over the vocabulary that sums to 1.
So in essence, temperature acts as a scaling factor on the log probability values before applying softmax.


### creative vs deterministic
- For creative writing, you may want lower Top-K/Top-P values along with a higher temperature to encourage more surprising and diverse generations.

- For analytical tasks where precision is crucial, higher Top-K/Top-P with lower temperature keeps the model focused.


#### This being a medical text analysis project, we need to be precise and determistic in the responses. Hence temperature is set to 0


### Default values of params in Llama cpp
- temperature	- Defaults to 0.8
- top_p	- Defaults to 0.95
- top_k	- Defaults to 40

### For this project we are using,
- temperature=0
- top_k=50


temperature=0,top_p=0.95,top_k=50

### Query: Describe the symptoms and treatment options for diabetes mellitus?

In [ ]:
user_input_ft1='Describe the symptoms and treatment options for diabetes mellitus?'
generate_rag_response(user_input_ft1)

Llama.generate: prefix-match hit


'Symptoms: The symptoms of diabetes mellitus include polydipsia, polyphagia, polyuria, orthostatic hypotension, dehydration, weakness, fatigue, mental status changes, weight loss, nausea and vomiting, blurred vision, and predisposition to bacterial or fungal infections. Patients may experience symptoms of hyperglycemia that come and go as plasma glucose levels fluctuate.\n\nTreatment options: The treatment for diabetes mellitus includes diet, exercise, and drugs that reduce glucose levels'

### Query: What are the diagnostic criteria for identifying Parkinson's disease?

In [ ]:
user_input_ft2="What are the diagnostic criteria for identifying Parkinson's disease?"
generate_rag_response(user_input_ft2)

Llama.generate: prefix-match hit


"The diagnostic criteria for identifying Parkinson's disease include:\n1. Characteristic unilateral resting tremor, decreased movement, or rigidity.\n2. The tremor disappears (or attenuates) during finger-to-nose coordination testing.\n3. Patients cannot perform rapidly alternating or rapid successive movements well.\n4. Sensation and strength are usually normal.\n5. Reflexes are normal but may be difficult to elicit due to tremor or rigidity.\n6. Patients may not suppress eye closure when the frontal muscle is tapped"

### Query: What are some common diseases and conditions?

In [ ]:
user_input_ft3="What are some common diseases and conditions?"
o_ft3=generate_rag_response(user_input_ft3)
o_ft3

Llama.generate: prefix-match hit


'Answer:\nHeart disease (see also Arrhythmias; Cardiovascular disease; Coronary artery disease; Heart failure; Myocardial infarction), congenital (2968), Lyme disease (1269, 1271), myocardial infarction (2114), Heartburn (125), rheumatic (2861, 2862), valvular (2179-2193), Heart failure (395, 2118-213'

- We will try with different params

In [ ]:
user_input_ft3="What are some common diseases and conditions?"
o_ft3_1=generate_rag_response(user_input_ft3,temperature=0.1, max_tokens=350)
o_ft3_1

Llama.generate: prefix-match hit


'Answer:\nHeart disease (see also Arrhythmias; Cardiovascular disease; Coronary artery disease; Heart failure; Myocardial infarction), congenital (2968), Lyme disease (1269, 1271), myocardial infarction (2114), Heartburn (125), rheumatic (2861, 2862), valvular (2179-2193), Heart failure (395, 2118-2131).'

In [ ]:
o_ft3_3=generate_rag_response(user_input_ft3,temperature=0, max_tokens=350)
o_ft3_3

Llama.generate: prefix-match hit


'Answer:\nHeart disease (see also Arrhythmias; Cardiovascular disease; Coronary artery disease; Heart failure; Myocardial infarction), congenital (2968), Lyme disease (1269, 1271), myocardial infarction (2114), Heartburn (125), rheumatic (2861, 2862), valvular (2179-2193), Heart failure (395, 2118-2131).'

In [ ]:
o_ft3_1==o_ft3_3

True

- There's no difference between temperature = 0  and temperature = 0.1, so we'll go with temperature = 0

In [ ]:
o_ft3_4=generate_rag_response(user_input_ft3,max_tokens=350,temperature=0, top_k=20)
o_ft3_4

Llama.generate: prefix-match hit


'Answer:\nHeart disease (see also Arrhythmias; Cardiovascular disease; Coronary artery disease; Heart failure; Myocardial infarction), congenital (2968), Lyme disease (1269, 1271), myocardial infarction (2114), Heartburn (125), rheumatic (2861, 2862), valvular (2179-2193), Heart failure (395, 2118-2131).'

In [ ]:
o_ft3_4==o_ft3_3

True

In [ ]:
o_ft3_5=generate_rag_response(user_input_ft3, temperature=0, top_k=1, max_tokens=256)
o_ft3_5

Llama.generate: prefix-match hit


'Answer:\nHeart disease (see also Arrhythmias; Cardiovascular disease; Coronary artery disease; Heart failure; Myocardial infarction), congenital (2968), Lyme disease (1269, 1271), myocardial infarction (2114), Heartburn (125), rheumatic (2861, 2862), valvular (2179-2193), Heart failure (395, 2118-2131).'

In [ ]:
user_input_ft4="Which therapeutic approaches are mentioned for managing cardiovascular diseases?"
o_ft4=generate_rag_response(user_input_ft4)
o_ft4


Llama.generate: prefix-match hit


'Answer: The mentioned therapeutic approaches for managing cardiovascular diseases include the use of β-blockers, nitroglycerin and Ca channel blockers for symptom control, revascularization if symptoms persist despite medical therapy, ACE inhibitors and statins, reversing risk factors such as smoking cessation and weight loss, aggressive reduction of total and LDL cholesterol, and an exercise program. Additionally, drugs like aspirin, clopidogrel, diltiazem, verapamil, carvedilol, and nitroglycerin are mentioned for their role'

In [ ]:
o_ft4_1=generate_rag_response(user_input_ft4, temperature=0, top_k=1, max_tokens=256)
o_ft4_1

Llama.generate: prefix-match hit


'Answer: The therapeutic approaches mentioned for managing cardiovascular diseases include the use of β-blockers to limit symptoms and prevent infarction and sudden death, antiplatelet drugs such as aspirin or clopidogrel to inhibit platelet aggregation, nitroglycerin as a potent smooth-muscle relaxant and vasodilator, aggressive reduction of total and LDL cholesterol, weight loss, and an exercise program. Additionally, hypertension is treated diligently and smokers are encouraged to stop smoking.'

How can the diagnosis of multiple sclerosis be confirmed?\n4. Name three medications used in the treatment of Parkinson's disease.\n5. Which vitamin deficiency is characterized by weakness, numbness, and tingling sensations in the hands and feet?\n6. What are the possible complications of untreated diabetes?

Llama.generate: prefix-match hit
1. What are some common methods used for laboratory diagnosis of infectious diseases according to The Merck Manual of Diagnosis & Therapy, 19th Edition?\n2. Can you list some examples of infectious diseases that can be diagnosed through laboratory tests mentioned in the manual?\n3. What role does a microbiologist play in diagnosing infectious diseases based on the information provided in The Merck Manual of Diagnosis & Therapy, 19th Edition?\n4. How does the manual suggest identifying the specific type or strain of a bacterial infection for effective treatment?\n5

In [ ]:
user_input_ft5="What are some common methods used for laboratory diagnosis of infectious diseases?"
o_ft5=generate_rag_response(user_input_ft4)
o_ft5


Llama.generate: prefix-match hit


'Answer: The therapeutic approaches mentioned for managing cardiovascular diseases include the use of β-blockers to limit symptoms and prevent infarction and sudden death, antiplatelet drugs such as aspirin or clopidogrel to inhibit platelet aggregation, nitroglycerin as a potent smooth-muscle relaxant and vasodilator, aggressive reduction of total and LDL cholesterol, weight loss, and an exercise program. Additionally, hypertension is treated diligently and smokers are encouraged to stop smoking.'

In [ ]:
o_ft5_1=generate_rag_response(user_input_ft5, temperature=0, top_k=1, max_tokens=256)
o_ft5_1

Llama.generate: prefix-match hit


'Based on the context, some common methods used for laboratory diagnosis of infectious diseases include microscopy, culture, immunologic tests (such as agglutination tests, enzyme immunoassays, Western blot, precipitation tests, and complement fixation tests), chromatographic methods, and nucleic acid-based identification methods. Microscopy can detect a large variety of pathogens but may require the use of specific stains or other identification methods for certain pathogens. Culture is the gold standard for identification of organisms but may not be available for days or weeks and may not be able to culture all pathogens. Immunologic tests identify antibodies to the organism, while chromatographic methods separate and identify microbial components or products using high-performance liquid chromatography or gas chromatography. Nucleic acid-based identification methods detect organism-specific DNA or RNA sequences extracted from the microorganism and are generally specific and highly s

In [ ]:
o_ft5_2=generate_rag_response(user_input_ft5, temperature=0, top_k=1, max_tokens=500)
o_ft5_2

Llama.generate: prefix-match hit


"Based on the context, some common methods used for laboratory diagnosis of infectious diseases include microscopy, culture, immunologic tests (such as agglutination tests, enzyme immunoassays, Western blot, precipitation tests, and complement fixation tests), chromatographic methods, and nucleic acid-based identification methods. Microscopy can be done quickly but accuracy depends on the experience of the microscopist and quality of equipment. Culture is normally the gold standard for identification of organisms but results may not be available for days or weeks, and not all pathogens can be cultured. Immunologic tests identify antibodies to the organism and can be useful when culture results are not available or when the pathogen cannot be cultured. Chromatographic methods separate and identify microbial components or products using high-performance liquid chromatography (HPLC) or gas chromatography, usually by comparison of an organism's fatty acids to a database. Nucleic acid-based

## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation. We illustrate this evaluation based on the answeres generated to the question from the previous section.

- We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

### Defining the Evaluation Prompts

In [ ]:
groundedness_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
"""

In [ ]:
relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
"""

In [ ]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [ ]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm_mistral(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]
    print(f"llm_response-{answer}")

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm_mistral(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = llm_mistral(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

### Query 1: Which disease is described as a chronic inflammatory disorder characterized by persistent pain, swelling, and joint damage?

In [ ]:
user_input_fn_1 = "Which disease is described as a chronic inflammatory disorder characterized by persistent pain, swelling, and joint damage?"
ground,rel = generate_ground_relevance_response(user_input_fn_1,max_tokens=600,top_k=1)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit


llm_response- Rheumatoid Arthritis


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the question and the context provided.
2. Understand the question, which is asking for a disease that is characterized by chronic inflammation, persistent pain, and joint damage.
3. Read through the context to identify any information related to such a disease.
4. Determine if the AI generated answer, "Rheumatoid Arthritis," can be derived solely from the context provided.

Explanation:
The question asks for a chronic inflammatory disorder characterized by persistent pain and joint damage. The context discusses Rheumatoid Arthritis (RA) as a chronic systemic autoimmune disease that primarily involves the joints, causes damage mediated by cytokines, chemokines, and metalloproteases, and leads to progressive destruction of articular structures. It also mentions that RA is a chronic inflammatory disorder. Therefore, the answer "Rheumatoid Arthritis" can be derived solely from the context provided.

Evaluation:
The metric is followed completely as

In [ ]:
user_input_fn_2 = "What are the symptoms and medications used in the treatment of Parkinson's disease.?"
ground,rel = generate_ground_relevance_response(user_input_fn_2,max_tokens=600,top_k=1)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit


llm_response- Parkinson's disease is a degenerative CNS disorder characterized by resting tremor, muscular rigidity, slow and decreased movement, and postural instability. The symptoms are treated with various medications including levodopa plus carbidopa, MAO-B inhibitors, anticholinergic drugs, dopamine agonists, and COMT inhibitors. Some oral drugs commonly used to relieve symptoms include clonazepam, valproate, levetiracetam, and 5-hydroxytryptophan with carbidopa. The Merck Manual suggests that early use of levodopa may hasten development of adverse effects and inconsistency of drug response, so some experts prefer to delay its use and try other medications first. Treatment also involves correction of underlying metabolic disturbances.


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the symptoms mentioned in the context related to Parkinson's disease.
2. Identify the medications mentioned in the context for treating Parkinson's disease symptoms.
3. Check if the AI generated answer includes both the symptoms and medications identified from the context.
4. Verify that the information provided in the AI generated answer is derived only from the context.

Explanation:
The AI generated answer correctly identifies the symptoms of Parkinson's disease as resting tremor, muscular rigidity, slow and decreased movement, and postural instability. It also lists various medications used for treating these symptoms including levodopa plus carbidopa, MAO-B inhibitors, anticholinergic drugs, dopamine agonists, COMT inhibitors, clonazepam, valproate, levetiracetam, and 5-hydroxytryptophan with carbidopa. These symptoms and medications are directly taken from the context.

Therefore, the metric is followed completely as the answer is derive

In [ ]:
user_input_fn_3 = "Which diseases are described as a chronic inflammatory disorder characterized by persistent pain, swelling?"
ground,rel = generate_ground_relevance_response(user_input_fn_3,max_tokens=900,top_k=1)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit


llm_response- Rheumatic diseases such as rheumatoid arthritis and gout are described as chronic inflammatory disorders characterized by persistent pain and swelling.


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the key information in the context related to the question.
2. Determine if the AI generated answer is derived only from the information presented in the context.
3. Compare the answer with the key information identified in step 1.

Explanation:
The context provides detailed information about chronic pain, its symptoms and signs, diagnosis, and treatment. Among these, the relevant information for the question are "Rheumatic diseases such as rheumatoid arthritis and gout are described as chronic inflammatory disorders" and "persistent pain and swelling." The AI generated answer matches this information exactly.

Evaluation:
The metric is followed completely.

Rating:
Based on the evaluation criteria, I would rate the answer a 5.

 Steps to evaluate the context as per the relevance metric:
1. Identify the main aspects of the question: The question asks for diseases that are described as a chronic inflammatory disorder characterized by persistent

In [ ]:
user_input_fn_4 = "What are the possible complications of untreated diabetes. Discuss possible treatments to cure?"
ground,rel = generate_ground_relevance_response(user_input_fn_4,max_tokens=900,top_k=1)

print(ground,end="\n\n")
print(rel)

Llama.generate: prefix-match hit


llm_response- The possible complications of untreated diabetes include various health issues affecting different body systems such as the eyes (diabetic retinopathy), kidneys (diabetic nephropathy), nerves (diabetic neuropathy), skin, rheumatologic diseases, ophthalmologic diseases, hepatobiliary diseases, and dermatologic diseases. Depression and dementia are also common.

As for treatments to cure diabetes, the context provided does not mention any specific cures. However, it emphasizes the importance of intensive control of plasma glucose levels to prevent or slow down the progression of these complications. Treatments for individual complications include argon laser photocoagulation or vitrectomy for diabetic retinopathy and strict glycemic control and early detection and treatment for diabetic nephropathy.


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 Steps to evaluate the answer:
1. Identify the information in the context related to possible complications of untreated diabetes and potential treatments.
2. Check if the AI generated answer only includes information derived from the context.
3. Evaluate the extent to which the answer adheres to the metric.

Explanation:
The AI generated answer mentions various health issues that can be complications of untreated diabetes, which are all taken directly from the context. It also correctly states that there is no mention of a cure for diabetes in the context and emphasizes the importance of intensive control of plasma glucose levels to prevent or slow down the progression of these complications. The answer also includes information about treatments for individual complications, which are also mentioned in the context.

Therefore, the answer adheres to the metric as it is derived only from the information presented in the context.

Rating:
Based on the evaluation criteria, I would rate th

## Recommendations

* Prompt Engineering improved overall accuracy of response
  * Well-crafted prompts: Well-crafted prompts lead to
more accurate, relevant, and high-quality AI outputs.
  * Enhanced Model Performance: Prompt engineering can improve the overall performance of AI models on specific tasks.
  * Greater Control over AI Behavior: By understanding how AI models respond to prompts, developers can exert greater control over their behavior

* The RAG-based output is superior to the generic LLM output because it provides a more context-aware and strategically relevant response. While the generic output lists common applications of LLMs, it lacks specificity and mainly presents a laundry list of use cases without much depth or insight.

* In contrast, the RAG output offers a more tailored and innovative perspective, discussing unique scenarios such as Private LLMs, Open Source LLMs, and Sovereign LLMs, with relevant examples like BloombergGPT and SoftBank's localized Japanese LLM.

* This makes the RAG output more credible and informative, as it connects specific examples to broader industry trends and motivations, such as the need for customizability, data sovereignty, and strategic localization.

* Additionally, the RAG output is more coherent and contextually connected, organizing information into meaningful categories that enhance the clarity and narrative flow. This results in a more engaging and valuable answer, providing deeper insights into the evolving landscape of LLM applications compared to the more informational and generic approach of the LLM output.

* LLM is used to evaluate RAG, for evaluating metrics such as groundedness and relevancy of the responses

* Vector database creation time increases with the number of pages in the PDF document.
* Retrieval parameter **`k`** is critical as the answer can be spread across multiple contexts.
* **`chunk_overlap`** (option that allows adjacent chunks to share certain common information) ensures coherence, especially when context spans across chunks. This is found to be critical in retrieving the coherent response
* **`max_tokens`** depends on query complexity; higher values yield detailed responses, while simple queries result in concise outputs despite large token limits due to prompt design and zero **`temperature`**.
* Refine prompt design and temperature settings to control response length and creativity.
* Continuously adjust RAG parameters based on specific use cases for optimal performance.
* Prioritize groundedness and relevance in evaluations to ensure reliable and contextually accurate outputs.
* Continually establish a feedback loop to fine-tune parameters, improving performance for diverse query types.
* Frameworks such as prompt flow, Lang Chain, Llama Index can be used for an end-end production ready AI applications.It provides a comprehensive solution that simplifies the process of prototyping, experimenting, iterating, and deploying your AI applications.

